In [18]:
import time
import datetime
import os
from Research.backtest.bt import BTDaily
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
import AlphaNet
from AlphaNet.Data import concat_original_data , generate_shift_data , generate_alpha_list
# read_task
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()
factor_info = task_info["Factor"]

In [22]:
alpha_list = factor_info.loc[16,"alpha_list"]

In [24]:
alpha_list = [i.replace("10","5") for i in alpha_list]
alpha_list

['COV_div_open_close_div_high_close_5',
 'COV_div_open_close_div_low_close_5',
 'COV_div_open_close_close_5',
 'COV_div_open_close_volume_sqrt_8_5',
 'COV_div_open_close_pct_chg_5',
 'COV_div_open_close_div_close_free_turnover_5',
 'COV_div_open_close_div_low_high_5',
 'COV_div_high_close_div_low_close_5',
 'COV_div_high_close_close_5',
 'COV_div_high_close_volume_sqrt_8_5',
 'COV_div_high_close_pct_chg_5',
 'COV_div_high_close_div_close_free_turnover_5',
 'COV_div_high_close_div_low_high_5',
 'COV_div_low_close_close_5',
 'COV_div_low_close_volume_sqrt_8_5',
 'COV_div_low_close_div_close_free_turnover_5',
 'COV_div_low_close_div_low_high_5',
 'COV_close_div_vwap_close_5',
 'COV_close_volume_sqrt_8_5',
 'COV_close_pct_chg_5',
 'COV_close_div_close_free_turnover_5',
 'COV_close_div_low_high_5',
 'COV_div_vwap_close_volume_sqrt_8_5',
 'COV_div_vwap_close_div_close_free_turnover_5',
 'COV_div_vwap_close_div_low_high_5',
 'COV_div_vwap_close_div_vwap_close_2_5',
 'COV_volume_sqrt_8_pct_chg

In [25]:
concat_original_data("AlphaNetV4_Moving5", alpha_list, start_date = "2016-01-01", end_date = "2021-06-01",
                     output_path="/home/wuwenjun/Data/", universe='Investable')

['COV_div_open_close_div_high_close_5', 'COV_div_open_close_div_low_close_5', 'COV_div_open_close_close_5', 'COV_div_open_close_volume_sqrt_8_5', 'COV_div_open_close_pct_chg_5', 'COV_div_open_close_div_close_free_turnover_5', 'COV_div_open_close_div_low_high_5', 'COV_div_high_close_div_low_close_5', 'COV_div_high_close_close_5', 'COV_div_high_close_volume_sqrt_8_5', 'COV_div_high_close_pct_chg_5', 'COV_div_high_close_div_close_free_turnover_5', 'COV_div_high_close_div_low_high_5', 'COV_div_low_close_close_5', 'COV_div_low_close_volume_sqrt_8_5', 'COV_div_low_close_div_close_free_turnover_5', 'COV_div_low_close_div_low_high_5', 'COV_close_div_vwap_close_5', 'COV_close_volume_sqrt_8_5', 'COV_close_pct_chg_5', 'COV_close_div_close_free_turnover_5', 'COV_close_div_low_high_5', 'COV_div_vwap_close_volume_sqrt_8_5', 'COV_div_vwap_close_div_close_free_turnover_5', 'COV_div_vwap_close_div_low_high_5', 'COV_div_vwap_close_div_vwap_close_2_5', 'COV_volume_sqrt_8_pct_chg_5', 'COV_volume_sqrt_8_tu

100%|██████████| 1310/1310 [01:00<00:00, 21.53it/s]


(                            COV_div_open_close_div_high_close_5  \
 timestamp           ticker                                        
 2016-03-14 09:30:00 000020                             0.003519   
                     600960                             0.003097   
                     000885                             0.003078   
                     603022                             0.002983   
                     000838                             0.002835   
 ...                                                         ...   
 2021-05-31 09:30:00 688551                            -0.000711   
                     300385                            -0.000796   
                     688689                            -0.001115   
                     300637                            -0.001163   
                     300257                            -0.001476   
 
                             COV_div_open_close_div_low_close_5  \
 timestamp           ticker                    

In [26]:
alpha_name = "AlphaNetV4_Moving5"
shift = 1
sequence = 20
generate_shift_data(alpha_name, shift,sequence, target = None, data_path="/home/wuwenjun/Data/")

100%|██████████| 1705/1705 [00:25<00:00, 65.94it/s] 


[19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


100%|██████████| 1291/1291 [21:15<00:00,  1.01it/s]


,,COV_div_open_close_div_high_close_5_Shift_19,COV_div_open_close_div_low_close_5_Shift_19,COV_div_open_close_close_5_Shift_19,COV_div_open_close_volume_sqrt_8_5_Shift_19,COV_div_open_close_pct_chg_5_Shift_19,COV_div_open_close_div_close_free_turnover_5_Shift_19,COV_div_open_close_div_low_high_5_Shift_19,COV_div_high_close_div_low_close_5_Shift_19,COV_div_high_close_close_5_Shift_19,COV_div_high_close_volume_sqrt_8_5_Shift_19,...,RETURN_div_open_close_5_Shift_0,RETURN_div_high_close_5_Shift_0,RETURN_div_low_close_5_Shift_0,RETURN_div_vwap_close_5_Shift_0,RETURN_volume_sqrt_8_5_Shift_0,RETURN_pct_chg_5_Shift_0,RETURN_div_close_free_turnover_5_Shift_0,RETURN_div_price_turnover_5_Shift_0,RETURN_div_low_high_5_Shift_0,RETURN_div_vwap_close_2_5_Shift_0
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2016-02-05 09:30:00,000001,0.000743,0.000183,-0.328205,-0.005325,-0.105278,14.608132,-0.000511,0.000119,-0.200347,-0.003307,...,-1.449477e+00,-0.756533,-29074050.0,-1.177049e+00,0.026550,2.938300e+10,-0.167326,0.019685,0.012573,-0.011503
2016-02-15 09:30:00,000001,0.000333,0.000148,-0.525332,-0.005684,-0.061182,17.238657,-0.000170,0.000088,-0.351596,-0.003691,...,3.003436e+08,-0.369959,299940288.0,8.854202e+07,-0.083555,-1.094242e+00,0.993810,-0.029767,0.033990,0.008918
2016-02-16 09:30:00,000001,0.000328,0.000101,-0.311013,-0.004818,-0.055421,15.388370,-0.000213,0.000084,-0.246143,-0.003634,...,-1.722960e+00,-0.713994,-81784408.0,-1.680929e+00,-0.049415,6.895000e+09,0.498469,0.031929,0.006854,-0.013277
2016-02-17 09:30:00,000001,0.000206,0.000092,-0.000827,-0.004587,-0.040362,13.803769,-0.000104,0.000075,-0.004179,-0.003420,...,-1.907767e+07,-0.751499,-8966462.0,-1.176964e+01,0.018793,4.681824e-01,-0.133214,0.001848,0.005113,-0.003328
2016-02-18 09:30:00,000001,0.000153,0.000134,-0.008488,-0.000476,-0.018324,0.972903,-0.000013,0.000087,-0.019153,-0.000100,...,-1.182347e+08,0.698272,-76422280.0,-4.050801e+07,0.099203,2.403600e+10,-0.516484,0.011940,-0.010500,-0.004058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-25 09:30:00,688981,0.000110,0.000097,-0.007377,0.000384,-0.020993,-0.024639,-0.000010,0.000067,-0.003886,0.000285,...,-3.090695e+00,-0.962156,-230359648.0,-1.963511e+00,-0.027721,3.356700e+10,0.253359,0.030935,-0.000542,-0.021385
2021-05-26 09:30:00,688981,0.000107,0.000102,-0.008025,0.000194,-0.020557,-0.012127,-0.000003,0.000062,-0.003091,0.000472,...,-2.336150e+00,-0.505130,-20665450.0,3.566052e+06,0.070373,9.434000e+09,-0.413594,0.008538,-0.000255,0.000358
2021-05-27 09:30:00,688981,0.000078,0.000028,-0.001148,0.000870,-0.011548,-0.050353,-0.000048,0.000035,-0.001694,0.001293,...,4.372247e+01,1.084625,36514268.0,4.750664e+00,0.072660,-4.102000e+09,-0.425872,-0.007915,-0.004356,0.005837


In [15]:
a = pd.read_parquet("/home/wuwenjun/Data/AlphaNetV4_Moving10/2016/20160118.par")
a.columns

Index(['COV_div_open_close_div_high_close_10',
       'COV_div_open_close_div_low_close_10', 'COV_div_open_close_close_10',
       'COV_div_open_close_volume_sqrt_8_10', 'COV_div_open_close_pct_chg_10',
       'COV_div_open_close_div_close_free_turnover_10',
       'COV_div_open_close_div_low_high_10',
       'COV_div_high_close_div_low_close_10', 'COV_div_high_close_close_10',
       'COV_div_high_close_volume_sqrt_8_10',
       ...
       'RETURN_div_price_turnover_10', 'RETURN_div_low_high_10',
       'RETURN_div_vwap_close_2_10', 'DECAY_div_high_close_10',
       'DECAY_div_low_close_10', 'DECAY_volume_sqrt_8_10',
       'DECAY_div_close_free_turnover_10', 'DECAY_div_price_turnover_10',
       'DECAY_div_low_high_10', 'DECAY_div_vwap_close_2_10'],
      dtype='object', length=103)

In [16]:
len(a.columns)

103

In [ ]:
factor_info = task_info["Factor"]
if task_info["CPU"] == False:
    raise ValueError("No CPU is avaliable")
factor_index = factor_info[factor_info["status"] == "waiting"].index[0]
alpha_name = factor_info.loc[factor_index,"Alpha_Name"]
shift = factor_info.loc[factor_index,"shift"]
sequence = factor_info.loc[factor_index,"sequence"]
target = factor_info.loc[factor_index,"target"]
LR = factor_info.loc[factor_index,"LR"]
epoch_num = factor_info.loc[factor_index,"epoch_num"]
alpha_list = factor_info.loc[factor_index,"alpha_list"]
universe = factor_info.loc[factor_index,"universe"]
t1 = (datetime.datetime.utcnow() + datetime.timedelta(hours=8)).strftime('%H:%M')
factor_info.loc[factor_index,"status"] = "Running: " +t1
task_info["CPU"] = False
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
print(factor_info.loc[factor_index,:])

output_path = "/home/wuwenjun/Data/"

if alpha_name not in os.listdir(output_path):
    concat_original_data(alpha_name=alpha_name,alpha_list=alpha_list,output_path=output_path,universe = universe)
generate_shift_data(alpha_name=alpha_name,shift=shift,sequence=sequence,target=target,data_path=output_path)

# Task Generation
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()
original = task_info["Task"]
# Timelist & Task
time_list = ["2019-01-01","2019-06-01","2020-01-01","2020-06-01","2021-01-01","2021-06-01"]
time_list2 = []
for i in range(len(time_list)-1):
    time_list2.append([time_list[i],time_list[i+1]])
time_list2 = pd.DataFrame(time_list2,columns=["start_date","end_date"])

task = pd.DataFrame([
                    alpha_name + "_Shift_%i_Sequence_%i_%s" % (shift,sequence,target),
                    sequence,LR,epoch_num,len(alpha_list),
                        ],index=["Alpha_Name","sequence","LR","epoch_num","feature_num"]).T
task['value']=1
time_list2['value']=1
task = pd.merge(task,time_list2,how='left',on='value')
del task['value']
task["status"] = "waiting"
task["description"] = [{
    "target" : target,
    "alpha_name" : alpha_name,
    "alpha_list" : alpha_list,
    "shift" : shift,
    "sequence" : sequence
} for i in range(len(task))]
task = pd.concat([original,task],axis=0)
task.reset_index(drop=True,inplace=True)
task.index.names = ["task_id"]
task_info["Task"] = task
factor_info = task_info["Factor"]
t2 = (datetime.datetime.utcnow() + datetime.timedelta(hours=8)).strftime('_%H:%M | %m-%d')
factor_info.loc[factor_index,"status"] = "Finish: " +t1+t2
task_info["CPU"] = True
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/Task.npy",task_info)